DGL 框架是由纽约大学和 AWS 工程师共同开发的开源框架，旨在为大家提供一个在图上进行深度学习的工具，帮助大家更高效的实现算法。

## 1.1 关于图的基本概念

图是用以表示实体及其关系的结构，记为 $G=(V,E)$ 。图由两个集合组成，一是节点的集合 $V$ ，一个是边的集合 $E$ 。 在边集 $E$ 中，一条边$ (u,v) $连接一对节点 $u$ 和 $v$ ，表明两节点间存在关系。关系可以是无向的， 如描述节点之间的对称关系；也可以是有向的，如描述非对称关系。例如，若用图对社交网络中人们的友谊关系进行建模，因为友谊是相互的，则边是无向的； 若用图对Twitter用户的关注行为进行建模，则边是有向的。图可以是 有向的 或 无向的 ，这取决于图中边的方向性。

图可以是 加权的 或 未加权的 。在加权图中，每条边都与一个标量权重值相关联。例如，该权重可以表示长度或连接的强度。

图可以是 同构的 或是 异构的 。在同构图中，所有节点表示同一类型的实体，所有边表示同一类型的关系。 例如，社交网络的图由表示同一实体类型的人及其相互之间的社交关系组成。

相对地，在异构图中，节点和边的类型可以是不同的。例如，编码市场的图可以有表示”顾客”、”商家”和”商品”的节点， 它们通过“想购买”、“已经购买”、“是顾客”和“正在销售”的边互相连接。二分图是一类特殊的、常用的异构图， 其中的边连接两类不同类型的节点。例如，在推荐系统中，可以使用二分图表示”用户”和”物品”之间的关系。

在多重图中，同一对节点之间可以有多条（有向）边，包括自循环的边。例如，两名作者可以在不同年份共同署名文章， 这就带来了具有不同特征的多条边。

## 1.2 图、节点和边
DGL使用一个唯一的整数来表示一个节点，称为点$ID$；并用对应的两个端点ID表示一条边。同时，DGL也会根据边被添加的顺序， 给每条边分配一个唯一的整数编号，称为边ID。节点和边的ID都是从0开始构建的。在DGL的图里，所有的边都是有方向的， 即边 $(u,v)$ 表示它是从节点 $u$ 指向节点 $v$ 的。

对于多个节点，DGL使用一个一维的整型张量（如，PyTorch的Tensor类，TensorFlow的Tensor类或MXNet的ndarray类）来保存图的点ID， DGL称之为”节点张量”。为了指代多条边，DGL使用一个包含2个节点张量的元组$(U,V)$ ，其中，用 $(U[i],V[i])$ 指代一条 $U[i]$ 到 $V[i]$ 的边。

创建一个 DGLGraph 对象的一种方法是使用 dgl.graph() 函数。它接受一个边的集合作为输入。DGL也支持从其他的数据源来创建图对象。

下面的代码段使用了 dgl.graph() 函数来构建一个 DGLGraph 对象，对应着下图所示的包含4个节点的图。 其中一些代码演示了查询图结构的部分API的使用方法。

![](https://data.dgl.ai/asset/image/user_guide_graphch_1.png)

In [1]:
import dgl
import torch as th
# 边 0->1, 0->2, 0->3, 1->3
u, v = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
g = dgl.graph((u, v))
print(g) # 图中节点的数量是DGL通过给定的图的边列表中最大的点ID推断所得出的
# 获取节点的ID
print(g.nodes())
# 获取边的对应端点
print(g.edges())
# 获取边的对应端点和边ID
print(g.edges(form='all'))
# 如果具有最大ID的节点没有边，在创建图的时候，用户需要明确地指明节点的数量。
g = dgl.graph((u, v), num_nodes=8)

Using backend: pytorch


Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})
tensor([0, 1, 2, 3])
(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]))
(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]), tensor([0, 1, 2, 3]))


对于无向的图，用户需要为每条边都创建两个方向的边。可以使用 `dgl.to_bidirected()` 函数来实现这个目的。 如下面的代码段所示，这个函数可以把原图转换成一个包含反向边的图。

In [2]:
bg = dgl.to_bidirected(g)
bg.edges()

(tensor([0, 0, 0, 1, 1, 2, 3, 3]), tensor([1, 2, 3, 0, 3, 0, 0, 1]))

DGL支持使用 32 位或 64 位的整数作为节点ID和边ID。节点和边ID的数据类型必须一致。如果使用 64 位整数， DGL可以处理最多$2^{63}$−1 个节点或边。不过，如果图里的节点或者边的数量小于 $2^{63}$−1 ，用户最好使用 32 位整数。 这样不仅能提升速度，还能减少内存的使用。DGL提供了进行数据类型转换的方法，如下例所示。

In [3]:
edges = th.tensor([2, 5, 3]), th.tensor([3, 5, 0])  # 边：2->3, 5->5, 3->0
g64 = dgl.graph(edges)  # DGL默认使用int64
print(g64.idtype)
g32 = dgl.graph(edges, idtype=th.int32)  # 使用int32构建图
g32.idtype
g64_2 = g32.long()  # 转换成int64
g64_2.idtype
g32_2 = g64.int()  # 转换成int32
g32_2.idtype

torch.int64


torch.int32

## 1.3 节点和边的特征

DGLGraph 对象的节点和边可具有多个用户定义的、可命名的特征，以储存图的节点和边的属性。 通过 ndata 和 edata 接口可访问这些特征。 例如，以下代码创建了2个节点特征（分别在第8、15行命名为 'x' 、 'y' ）和1个边特征（在第9行命名为 'x' ）。

In [5]:
import dgl
import torch as th
g = dgl.graph(([0, 0, 1, 5], [1, 2, 2, 0])) # 6个节点，4条边
g

Graph(num_nodes=6, num_edges=4,
      ndata_schemes={}
      edata_schemes={})

In [6]:
g.ndata['x'] = th.ones(g.num_nodes(), 3)               # 长度为3的节点特征
g.edata['x'] = th.ones(g.num_edges(), dtype=th.int32)  # 标量整型特征
g

Graph(num_nodes=6, num_edges=4,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(), dtype=torch.int32)})

In [7]:
# 不同名称的特征可以具有不同形状
g.ndata['y'] = th.randn(g.num_nodes(), 5)

In [8]:
g.ndata['x'][1]                  # 获取节点1的特征

tensor([1., 1., 1.])

In [9]:
g.edata['x'][th.tensor([0, 3])]  # 获取边0和3的特征

tensor([1, 1], dtype=torch.int32)

关于 ndata 和 edata 接口的重要说明：

- 仅允许使用数值类型（如单精度浮点型、双精度浮点型和整型）的特征。这些特征可以是标量、向量或多维张量。

- 每个节点特征具有唯一名称，每个边特征也具有唯一名称。节点和边的特征可以具有相同的名称（如上述示例代码中的 'x' ）。

- 通过张量分配创建特征时，DGL会将特征赋给图中的每个节点和每条边。该张量的第一维必须与图中节点或边的数量一致。 不能将特征赋给图中节点或边的子集。

- 相同名称的特征必须具有相同的维度和数据类型。

- 特征张量使用”行优先”的原则，即每个行切片储存1个节点或1条边的特征。

- 对于加权图，用户可以将权重储存为一个边特征，如下。

对于加权图，用户可以将权重储存为一个边特征，如下:

In [10]:
# 边 0->1, 0->2, 0->3, 1->3
edges = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
weights = th.tensor([0.1, 0.6, 0.9, 0.7])  # 每条边的权重
g = dgl.graph(edges)
g.edata['w'] = weights  # 将其命名为 'w'
g

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.float32)})

## 1.4 从外部源创建图
### 从外部库创建图
以下代码片段为从SciPy稀疏矩阵和NetworkX图创建DGL图的示例。

In [11]:
import dgl
import torch as th
import scipy.sparse as sp
spmat = sp.rand(100, 100, density=0.05) # 5%非零项
dgl.from_scipy(spmat)                   # 来自SciPy
import networkx as nx
nx_g = nx.path_graph(5) # 一条链路0-1-2-3-4
dgl.from_networkx(nx_g) # 来自NetworkX

Graph(num_nodes=5, num_edges=8,
      ndata_schemes={}
      edata_schemes={})

注意，当使用 nx.path_graph(5) 进行创建时， DGLGraph 对象有8条边，而非4条。 这是由于 nx.path_graph(5) 构建了一个无向的NetworkX图 networkx.Graph ，而 DGLGraph 的边总是有向的。 所以当将无向的NetworkX图转换为 DGLGraph 对象时，DGL会在内部将1条无向边转换为2条有向边。 使用有向的NetworkX图 networkx.DiGraph 可避免该情况。

In [12]:
nxg = nx.DiGraph([(2, 1), (1, 2), (2, 3), (0, 0)])
dgl.from_networkx(nxg)

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})

## 从磁盘加载图
- 逗号分隔值（CSV）
- JSON/GML 格式
- DGL 二进制格式

具体使用可以参照：https://docs.dgl.ai/guide_cn/graph-external.html#id3

## 1.5 异构图

相比同构图，异构图里可以有不同类型的节点和边。这些不同类型的节点和边具有独立的ID空间和特征。 例如在下图中，”用户”和”游戏”节点的ID都是从0开始的，而且两种节点具有不同的特征。
![](https://data.dgl.ai/asset/image/user_guide_graphch_2.png)
个异构图示例。该图具有两种类型的节点(“用户”和”游戏”)和两种类型的边(“关注”和”玩”)。

### 创建异构图
在DGL中，一个异构图由一系列子图构成，一个子图对应一种关系。每个关系由一个字符串三元组 定义 (源节点类型, 边类型, 目标节点类型) 。由于这里的关系定义消除了边类型的歧义，DGL称它们为规范边类型。

下面的代码是一个在DGL中创建异构图的示例。

In [14]:
import dgl
import torch as th
# 创建一个具有3种节点类型和3种边类型的异构图
graph_data = {
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}
g = dgl.heterograph(graph_data)


In [15]:
g.ntypes# 节点类型

['disease', 'drug', 'gene']

In [16]:
g.etypes # 边的类型

['interacts', 'interacts', 'treats']

In [17]:
g.canonical_etypes

[('drug', 'interacts', 'drug'),
 ('drug', 'interacts', 'gene'),
 ('drug', 'treats', 'disease')]

与异构图相关联的 metagraph 就是图的模式。它指定节点集和节点之间的边的类型约束。 metagraph 中的一个节点 $u$ 对应于相关异构图中的一个节点类型。 metagraph 中的边 $(u,v)$ 表示在相关异构图中存在从 $u$ 型节点到 $v $型节点的边。

In [20]:
g

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])

In [19]:
g.metagraph().edges()

OutMultiEdgeDataView([('drug', 'drug'), ('drug', 'gene'), ('drug', 'disease')])

### 使用多种类型
当引入多种节点和边类型后，用户在调用DGLGraph API以获取特定类型的信息时，需要指定具体的节点和边类型。此外，不同类型的节点和边具有单独的ID。

In [21]:
# 获取图中所有节点的数量
g.num_nodes()


10

In [22]:
# 获取drug节点的数量
g.num_nodes('drug')

3

In [25]:
# 不同类型的节点有单独的ID。因此，没有指定节点类型就没有明确的返回值。
# g.nodes()

In [24]:
g.nodes('drug')

tensor([0, 1, 2])

为了设置/获取特定节点和边类型的特征，DGL提供了两种新类型的语法： g.nodes[‘node_type’].data[‘feat_name’] 和 g.edges[‘edge_type’].data[‘feat_name’] 。

In [26]:
# 设置/获取"drug"类型的节点的"hv"特征
g.nodes['drug'].data['hv'] = th.ones(3, 1)
g.nodes['drug'].data['hv']


tensor([[1.],
        [1.],
        [1.]])

In [27]:
# 设置/获取"treats"类型的边的"he"特征
g.edges['treats'].data['he'] = th.zeros(1, 1)
g.edges['treats'].data['he']

tensor([[0.]])